---
## 0. Импорты, скачивания

In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129
%pip install diskcache requests feedparser pandas transformers sentence_transformers symspellpy faiss-cpu sentencepiece accelerate

Looking in indexes: https://download.pytorch.org/whl/cu129
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import diskcache
import functools
import logging
import requests
from typing import TypedDict
from urllib.parse import urlencode
import feedparser
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from symspellpy import SymSpell, Verbosity
import pkg_resources


log = logging.getLogger("lab-3")
log.setLevel(logging.INFO)

disk_cache = diskcache.Cache("cache")


def cache(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = (func.__name__, args, frozenset(kwargs.items()))
        if key in disk_cache:
            logging.info(f"Cache hit: {key}")
            return disk_cache[key]
        result = func(*args, **kwargs)
        disk_cache[key] = result
        return result
    return wrapper

C:\Users\HuTao1Love\PycharmProjects\data-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HuTao1Love\AppData\Local\Temp\ipykernel_15352\3637700174.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


---
## 1. Выбрать тематику для поиска (например, “machine learning”, https://arxiv.org/list/cs.LG/recent) и использовать arXiv API для извлечения статей. Выгрузить названия, авторов и аннотации статей.


In [3]:
# Config
BASE_URL = "https://export.arxiv.org/api/query"
CATEGORY = "cs.FL"  # Formal Languages and Automata Theory
RESULTS_COUNT = 500
CHUNK_SIZE = 400
CHUNK_OVERLAP = 50

In [4]:
class Publication(TypedDict):
    title: str
    authors: list[str]
    summary: str
    url: str



@cache
def fetch_arxiv(category: str, results_count: int) -> list[Publication]:
    args = {
        "search_query": f"cat:{category}",
        "sortBy": "submittedDate",
        "sortOrder": "descending",
        "max_results": results_count
    }

    response = requests.get(f"{BASE_URL}?{urlencode(args)}")
    response.raise_for_status()

    feed = feedparser.parse(response.text)

    return [
        Publication(
            title=entry.title,
            authors=[author.name for author in entry.authors],
            summary=entry.summary,
            url=entry.id.replace('abs', 'pdf'),
        )
        for entry in feed.entries
    ]

papers = fetch_arxiv(CATEGORY, RESULTS_COUNT)
papers

[{'title': 'MightyPPL: Verification of MITL with Past and Pnueli Modalities',
  'authors': ['Hsi-Ming Ho',
   'Shankara Narayanan Krishna',
   'Khushraj Madnani',
   'Rupak Majumdar',
   'Paritosh Pandya'],
  'summary': 'Metric Interval Temporal Logic (MITL) is a popular formalism for specifying\nproperties of reactive systems with timing constraints. Existing approaches to\nusing MITL in verification tasks, however, have notable drawbacks: they either\nsupport only limited fragments of the logic or allow for only incomplete\nverification. This paper introduces MightyPPL, a new tool for translating\nformulae in Metric Interval Temporal Logic with Past and Pnueli modalities\n(MITPPL) over the pointwise semantics into timed automata. MightyPPL enables\nsatisfiability and model checking of a much more expressive specification logic\nover both finite and infinite words and incorporates a number of performance\noptimisations, including a novel symbolic encoding of transitions and a\nsymmetr

In [5]:
df = pd.DataFrame(papers)
df.sample(10, random_state=42)

,title,authors,summary,url
361,Robust Probabilistic Model Checking with Conti...,"[Xiaotong Ji, Hanchun Wang, Antonio Filieri, I...",Probabilistic model checking traditionally ver...,http://arxiv.org/pdf/2502.04530v1
73,Box-Reachability in Vector Addition Systems,"[Shaull Almagor, Itay Hasson, Michał Pilipczuk...",We consider a variant of reachability in Vecto...,http://arxiv.org/pdf/2508.12853v1
374,Fine-Grained Complexity of Ambiguity Problems ...,"[Karolina Drabik, Anita Dürr, Fabian Frei, Fil...","In the field of computational logic, two class...",http://arxiv.org/pdf/2501.14725v3
155,Measure-Theoretic Aspects of Star-Free and Gro...,"[Ryoma Sin'ya, Takao Yuyama]",A language $L$ is said to be ${\cal C}$-measur...,http://arxiv.org/pdf/2506.14134v1
104,Orchestration of Music by Grammar Systems,"[Jozef Makiš, Alexander Meduna, Zbyněk Křivka]",This application-oriented study concerns compu...,http://arxiv.org/pdf/2507.15314v1
394,Neural Cellular Automata and Deep Equilibrium ...,[Zhibai Jia],This essay discusses the connections and diffe...,http://arxiv.org/pdf/2501.03573v1
377,Infinite Time Turing Machines and their Applic...,"[Rukmal Weerawarana, Maxwell Braun]",This work establishes a rigorous theoretical f...,http://arxiv.org/pdf/2506.05351v1
124,Addition Automata and Attractors of Digit Syst...,"[Anjelo Gabriel R. Cruz, Manuel Joseph C. Loqu...",Let $A$ be an expanding $2 \times 2$ matrix wi...,http://arxiv.org/pdf/2507.06158v1
68,List of Results on the Černý Conjecture and Re...,[Mikhail V. Volkov],We survey results in the literature that estab...,http://arxiv.org/pdf/2508.15655v3
450,Can adversarial attacks by large language mode...,"[Manuel Cebrian, Andres Abeliuk, Jan Arne Telle]",Attributing outputs from Large Language Models...,http://arxiv.org/pdf/2411.08003v3


---
## 2. Разбиение на чанки. Разделить текст аннотаций на чанки по 500 слов с перекрытием в 50 слов для последующей обработки (размер чанков и перекрытий можно скорректировать).


In [6]:
def split_into_chunks(text, chunk_size, overlap):
    words = text.split()
    if len(words) <= chunk_size:
        return [text]

    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunks.append(" ".join(words[start:end]))
        if end >= len(words):
            break
        start = end - overlap

    return chunks


df["chunks"] = df["summary"].apply(lambda txt: split_into_chunks(txt, CHUNK_SIZE, CHUNK_OVERLAP))

example = df.iloc[52]
print(example["title"])
example["summary"]

QIP $ \subseteq $ AM(2QCFA)


'The class of languages having polynomial-time classical or quantum\ninteractive proof systems ($\\mathsf{IP}$ or $\\mathsf{QIP}$, respectively) is\nidentical to $\\mathsf{PSPACE}$. We show that $\\mathsf{PSPACE}$ (and so\n$\\mathsf{QIP}$) is subset of $\\mathsf{AM(2QCFA)}$, the class of languages\nhaving Arthur-Merlin proof systems where the verifiers are two-way finite\nautomata with quantum and classical states (2QCFAs) communicating with the\nprovers classically. Our protocols use only rational-valued quantum transitions\nand run in double-exponential expected time. Moreover, the member strings are\naccepted with probability 1 (i.e., perfect-completeness).'

In [7]:
chunk_rows = []
for _, row in df.iterrows():
    for i, chunk in enumerate(row["chunks"]):
        chunk_rows.append({
            "title": row["title"],
            "chunk_index": i,
            "chunk_text": chunk
        })

df_chunks = pd.DataFrame(chunk_rows)
df_chunks.head(5)

,title,chunk_index,chunk_text
0,MightyPPL: Verification of MITL with Past and ...,0,Metric Interval Temporal Logic (MITL) is a pop...
1,Cobham's theorem for the Gaussian integers,0,"Assuming the four exponentials conjecture, Han..."
2,Black-box Context-free Grammar Inference for R...,0,Black-box context-free grammar inference is cr...
3,Group Actions and Some Combinatorics on Words ...,0,We introduce generalizations of powers and fac...
4,"Balanced Fibonacci word rectangles, and beyond",0,"Following a recent paper of Anselmo et al., we..."


---
## 3. Векторизация текста. Использовать модель эмбеддингов (например, ‘all-MiniLM-L6-v2’) для преобразования текстов в векторное представление. Построить векторное хранилище с помощью FAISS.


In [8]:
class Vectorizer:
    def __init__(self, model='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model)

    def encode_texts(self, texts):
        return self.model.encode(texts)

In [9]:
class VectorStore:
    def __init__(self, dimension=384):
        self.index = faiss.IndexFlatIP(dimension)
        self.metadata = []

    def add_chunks(self, embeddings, metadata):
        embeddings = np.array(embeddings).astype('float32')
        self.index.add(embeddings)
        self.metadata.extend(metadata)

    def search(self, query_embedding, top_k=5):
        distances, indices = self.index.search(query_embedding.reshape(1, -1), top_k)
        results = []
        for i, (d, idx) in enumerate(zip(distances[0], indices[0])):
            meta = self.metadata[idx].copy()
            meta["similarity"] = float(d)
            meta["rank"] = i + 1
            results.append(meta)
        return results

In [10]:
class ArticleProcessor:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.vectorizer = Vectorizer(model_name)
        self.store = VectorStore()

    def process(self, df_chunks):
        texts = df_chunks["chunk_text"].tolist()
        embeddings = self.vectorizer.encode_texts(texts)

        metadata = df_chunks.to_dict(orient="records")
        self.store.add_chunks(embeddings, metadata)

    def search(self, query, top_k=5):
        query_emb = self.vectorizer.model.encode(query)
        return self.store.search(query_emb, top_k)


In [11]:
processor = ArticleProcessor()
processor.process(df_chunks)

---
## 4. Интеграция с LLM. Настроить взаимодействие с LLM (например, FLAN-T5 или OpenAI GPT). Использовать промпт для получения ключевых идей статьи или ответа на конкретный вопрос пользователя.


In [12]:
class Llm:
    def __init__(self, model="google/flan-t5-xl"):
        self.tokenizer = T5Tokenizer.from_pretrained(model)
        self.model = T5ForConditionalGeneration.from_pretrained(model)
        self.model.to("cuda")

    def generate_response(self, prompt, max_new_tokens=300):
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
        outputs = self.model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.2,
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def generate_response_with_template(self, prompt, metadata, max_new_tokens=300):
        context = '\n---\n'.join([
            f"Title: {m['title']} (Chunk {m.get('chunk_index', 0)})\n{m['chunk_text']}"
            for m in metadata
        ])

        full_prompt = (
            f"You are an expert in computer science theory.\n"
            f"Answer the question using only the information provided in the context.\n"
            f"Question: {prompt}\n\n"
            f"Context:\n{context}\n\n"
            f"Answer concisely and clearly:"
        )

        return self.generate_response(full_prompt, max_new_tokens)

---
## 5. Поиск по корпусу. Настроить промпт-инжиниринг для извлечения ключевых идей и результатов из текста. Добавить обработку опечаток в пользовательских запросах с использованием библиотек типа SymSpell или rapidfuzz. Протестируйте полученный результат. Рекомендуется привести в пример не менее пяти промптов. На примерах покажите, что RAG действительно улучшает качество ответов LLM.

In [13]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, 0, 1)
sym_spell.create_dictionary_entry("nfa", 1000)
sym_spell.create_dictionary_entry("dfa", 1000)

def correct_query(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term if suggestions else query


In [14]:
llm = Llm()

def get_response(llm, query):
    return llm.generate_response(query)

def get_response_with_RAG(llm, query):
    metadata = processor.search(query, top_k=3)
    return llm.generate_response_with_template(query, metadata)

def compare(llm, query):
    print(f"Query: {query}\n")
    print("Without RAG:\n", get_response(llm, query), "\n")
    print("With RAG:\n", get_response_with_RAG(llm, query))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 24.20it/s]


In [15]:
queries = [
    "Wht is the differece between determinisitc (DFA) and nondeterministic (NFA) automata?",
    "What is the inital and final state?",
    "how does a dfa procces a strng?",
    "how to minmize a dfa?",
    "How does a finite automaton proces a string?",
    "Can the same NFA be convertd to a DFA?",
    "How is a regular expression related to DFA/NFA?"
]

for q in queries:
    compare(llm, correct_query(q))
    print("\n", "="*80, "\n")

Query: what is the difference between deterministic dfa and no deterministic nfa automate



Token indices sequence length is longer than the specified maximum sequence length for this model (695 > 512). Running this sequence through the model will result in indexing errors


Without RAG:
 dfa is a deterministic arithmetic arithmetic automaton 

With RAG:
 a repetitive NFAwtw need not halt immediately, accepting or rejecting, but it may change into another state and continue with its computation


Query: what is the initial and final state

Without RAG:
 a molecule of water 

With RAG:
 (4, 0, 0) -> (2, 0, 0) -> (1, 0, 0), with all higher digit positions degenerating to the absorbing state (0, 0, 0).


Query: how does a dfa prices a strong

Without RAG:
 currency 

With RAG:
 reversible automata with multiple initial states (MRFA) recognize strictly more languages than sweeping reversible automata (sRFA), which are in turn stronger than one-way reversible automata with a single initial state (1RFA). The latter recognize strictly more languages than one-way permutation automata (1PerFA).


Query: how to minimize a dfa

Without RAG:
 dfas are a type of derivatives contract that allows a borrower to borrow money from a lender at a lower interest rate than a ba